# Q4-2.シャッターカメラ

## 問題
  **Q4-2.キーボードで特定のキーを入力した際に写真を撮影して保存するアプリを作りましょう。**

### ライブラリのインストール
今回はipywidetsを使うので、まずは下記のセルを実行してipywidgetsをインストールします。

In [ ]:
!pip install ipywidgets

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# opencvのライブラリをインポート
import cv2
# osのインポート
import os
# OAK-Dに必要なライブラリをインポート
import depthai as dai
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Image
import threading
# OAK-Dのパイプライン作成
pipeline = dai.Pipeline()

# ソースとアウトプットの設定
cam_rgb = pipeline.createColorCamera()

# preview size640x480に指定
cam_rgb.setPreviewSize(640, 480) 
cam_rgb.setInterleaved(False)

# ストリーミング名設定
xout_rgb = pipeline.createXLinkOut()
xout_rgb.setStreamName("rgb")
cam_rgb.preview.link(xout_rgb.input)

# webカメラの設定
DEVICE_ID = 0


def take_photo(name: str):
    """
    写真を撮って保存する関数。
    保存先はoutputディレクトリ以下。
    Args:
        name (str): 保存する画像ファイル名
    Returns:
        Bool: True なら成功, False なら失敗.

    """
    # ディスプレイを設定
    display_handle=display(None, display_id=True)
    # デバイスをパイプラインに接続
    with dai.Device(pipeline) as device:
        # フレームを取得して表示
        video = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
        frame = video.get().getCvFrame()
        ret, jpg = cv2.imencode('.jpeg', frame)


    # カメラ画像の取得
    # 取得に失敗した場合、Falseを返す
    if not ret:
        print('画像取得に失敗しました。')
        return False
    else:
        img = Image(data=jpg.tobytes())
        display_handle.update(img)
        file_name = 'output/' + name + '.jpg'
        cv2.imwrite(file_name, frame)
        print('ファイル名' + file_name + 'を保存しました。')
        return True


# 'output'というディレクトリを作成
os.makedirs('output', exist_ok=True)

### ここから問題  ###

"""(特定の文字入力で、写真を撮影して保存するコードを記述)"""

### ここまで  ###

### ヒント
カメラ画像を記録する関数take_photo()が既に作成されています。  
キーボードで正しい入力が得られた時に、この関数を呼び出すようにします。

**input()を使うとキーボード入力が取得できます。**  

以下のセルのように記述すると、キーボードで入力した文字列をvalに取得し、コマンドラインに同じ文字列を出力することができます。  
実行して確認してみましょう。

In [ ]:
val = input()
print(val)

**zfill()を使って、数値をゼロ埋めした文字列をつくることができます**

写真を保存する時は、1枚目は001,2枚目は002...というファイル名を付けたいので、引数でそのように渡してあげる必要があります。  
数値をゼロ埋めした文字列にするには、_zfill()_ を使います。  

下記のセルを実行して確認してみましょう。  
zfillの引数はゼロ埋めする桁数です。  

In [ ]:
num = 1
# numを3桁にゼロ埋めした文字列にする
num_after = str(num).zfill(3)
print(num_after)

完成したら実行してみましょう。  
指定したキーを押した際に写真が撮影できるようになれば成功です。

## 回答と解説
以下は回答例と解説です。動作が正しければ、これと同じでなくても問題ありません。

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# opencvのライブラリをインポート
import cv2
# osのインポート
import os
# OAK-Dに必要なライブラリをインポート
import depthai as dai
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Image
import threading
# OAK-Dのパイプライン作成
pipeline = dai.Pipeline()

# ソースとアウトプットの設定
cam_rgb = pipeline.createColorCamera()

# preview size640x480に指定
cam_rgb.setPreviewSize(640, 480) 
cam_rgb.setInterleaved(False)

# ストリーミング名設定
xout_rgb = pipeline.createXLinkOut()
xout_rgb.setStreamName("rgb")
cam_rgb.preview.link(xout_rgb.input)

# webカメラの設定
DEVICE_ID = 0


def take_photo(name: str):
    """
    写真を撮って保存する関数。
    保存先はoutputディレクトリ以下。
    Args:
        name (str): 保存する画像ファイル名
    Returns:
        Bool: True なら成功, False なら失敗.

    """
    # ディスプレイを設定
    display_handle=display(None, display_id=True)
    # デバイスをパイプラインに接続
    with dai.Device(pipeline) as device:
        # フレームを取得して表示
        video = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
        frame = video.get().getCvFrame()
        ret, jpg = cv2.imencode('.jpeg', frame)


    # カメラ画像の取得
    # 取得に失敗した場合、Falseを返す
    if not ret:
        print('画像取得に失敗しました。')
        return False
    else:
        img = Image(data=jpg.tobytes())
        display_handle.update(img)
        file_name = 'output/' + name + '.jpg'
        cv2.imwrite(file_name, frame)
        print('ファイル名' + file_name + 'を保存しました。')
        return True


# 'output'というディレクトリを作成
os.makedirs('output', exist_ok=True)

### ここから問題  ###

# ファイル名の通し番号
num = 0
# Ctrl + Cで終了するまでループし続ける
while(True):
    print('cを入力後エンターを押すと写真を撮影します。')
    # キーボード入力待ち
    val = input()
    # cが入力されていた場合写真を撮る。
    if(val == 'c'):
        # numを3桁に0埋めしてtake_photo()に渡す。
        take_photo(str(num).zfill(3))
        # 写真を撮ったらnumを+1する。
        num += 1

### ここまで  ###

#### 74行目:
```python
num = 0
```
ファイル名の通し番号として使う変数を最初に作っておきます。

#### 76行目:
```python
while(True):
```
while()でループさせます。  
条件式はTrueなので、終了するまでループし続けます。

#### 77行目:
```python
print('cを入力後エンターを押すと写真を撮影します。')

```
操作説明文をコマンドラインに出力します。  
今回は'c'が入力されたら写真を撮る、としました。

#### 79行目:
```python
val = input()
```
input()関数を使ってキーボードの入力待ちをします。

#### 81行目:
```python
if(val == 'c'):
```
キーボードで入力された値が'c'かどうかを判定します。  

今回はelse文を書いていません。  
'c'以外だった場合は、ループの始めに戻って再びキーボード入力待ちとなります。  

#### 83行目:
```python
take_photo(str(num).zfill(3))
```
take_photo()関数で写真を撮影します。  
引数は、numを文字列に変換して3桁でゼロ埋めした文字列です。  

#### 85行目:
```python
num += 1
```
numを一つ進めておきます。  
こうすることで、次の撮影時は1つ先の番号で名前付けがされます。  

## 最後に
これでQ4-2は終了です。今までの練習問題を参考にオリジナルソフト開発に取り組んでみましょう。